In [1]:
## download praw 
!pip install praw
!pip install --upgrade https://github.com/praw-dev/praw/archive/master.zip
import praw
import pandas as pd

  Using cached praw-7.8.1-py3-none-any.whl.metadata (9.4 kB)
  Using cached prawcore-2.4.0-py3-none-any.whl.metadata (5.0 kB)
  Using cached update_checker-0.18.0-py3-none-any.whl.metadata (2.3 kB)
Using cached praw-7.8.1-py3-none-any.whl (189 kB)
Using cached prawcore-2.4.0-py3-none-any.whl (17 kB)
Using cached update_checker-0.18.0-py3-none-any.whl (7.0 kB)
  Using cached https://github.com/praw-dev/praw/archive/master.zip
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for praw: filename=praw-7.8.2.dev0-py3-none-any.whl size=189388 sha256=afe901dd664f0ff94028d9ea2f6afad4fa8917afe32240cffee47ad3913baf0e
  Stored in directory: /tmp/pip-ephem-wheel-cache-y9j6dtx4/wheels/f5/5b/3e/81dcf511298e5fcd6a4147a8f9c19568acb59be4a72eef36ba
Successfully built praw
  Attempting uninstall: praw
    Found existing installation: praw 7.8.1
    Uninstalling p

In [2]:
# Set up Reddit API client using praw
reddit = praw.Reddit(
    client_id='dc3EBg9kKK2_OyV5xWqJXQ',  # Replace with your client_id
    client_secret='HlKaJW1s-jdGR5u5eXbihDg6FE9Lbg',  # Replace with your client_secret
    user_agent='seniorcomps',  # Replace with your user_agent
    username='rbeal@oxy.edu',  # Replace with your Reddit username (optional)
)

def scrape_subreddit_to_csv(subreddit_name, exclude_words, post_limit=50, replies_limit=5, output_file="reddit_data.csv"):
    """
    Scrapes subreddit posts and top replies, excluding posts with specific words, pictures, or links,
    and saves the results to a CSV file.

    Args:
        subreddit_name (str): Name of the subreddit to scrape.
        exclude_words (list): List of words to exclude posts by.
        post_limit (int): Number of posts to retrieve.
        replies_limit (int): Number of top replies per post to retrieve.
        output_file (str): Name of the CSV file to save the data.
    """
    # List to store the data
    data = []

    # Access the subreddit
    subreddit = reddit.subreddit(subreddit_name)

    # Iterate over the subreddit posts
    for post in subreddit.new(limit=post_limit):
        # Filter out posts containing excluded words, images, or links
        if (any(word.lower() in post.title.lower() or word.lower() in post.selftext.lower() for word in exclude_words) 
                or not post.is_self  # Exclude non-text posts (images, links)
                or 'http' in post.selftext.lower()):  # Exclude posts containing links
            continue  # Skip posts that meet exclusion criteria

        # Iterate through post's top-level comments and get top replies
        post.comments.replace_more(limit=0)  # Load all top-level comments
        top_replies = post.comments[:replies_limit]
        
        # Collect post and replies data
        for reply in top_replies:
            data.append({
                "post_title": post.title,
                "post_body": post.selftext,
                "post_url": post.url,
                "post_score": post.score,
                "post_id": post.id,
                "post_author": str(post.author),
                "post_created": post.created_utc,
                "post_num_comments": post.num_comments,
                "reply_author": str(reply.author),
                "reply_body": reply.body,
                "reply_score": reply.score,
                "reply_created": reply.created_utc
            })
    
    # Save data to a CSV file using pandas
    df = pd.DataFrame(data)
    df.to_csv(output_file, index=False)
    print(f"Data saved to {output_file}")

# Example usage
excluded_words = ["biden", "trump", "harris", "election"]  # Words to exclude
subreddit_name = "librarians"  # Subreddit to scrape
output_file = "reddit_data.csv"
scrape_subreddit_to_csv(subreddit_name, excluded_words, post_limit=50, replies_limit=5, output_file=output_file)


Data saved to reddit_data.csv
